In [1]:
import os

In [2]:
!pwd

/home/jovyan/notebooks/executions


In [3]:
import sys
sys.path.append('/home/jovyan/')

In [4]:
from experiments.reporting.report_repository import ReportRepository
from experiments.slack_client import SlackNotifier
from neat.neat_logger import get_neat_logger
from neat.population_engine import EvolutionEngine
from neat.reporting.reports_pyneat import EvolutionReport
from neat.utils import timeit
import math
import random
from itertools import count
import numpy as np

from experiments.logger import logger
from experiments.slack_client import Notifier
from neat.configuration import get_configuration
from neat.evaluation.evaluation_engine import EvaluationStochasticEngine
from neat.evolution_operators.crossover import Crossover
from neat.evolution_operators.mutation import Mutation
from neat.genome import Genome
from neat.reporting.reports_pyneat import EvolutionReport
from neat.species import SpeciationEngine
from neat.stagnation import Stagnation
from neat.population_engine import PopulationEngine
from neat.utils import timeit
import neat
import tests
from config_files.configuration_utils import create_configuration

In [5]:
# dir(neat)

In [6]:
LOGS_PATH = f'{os.getcwd()}/'
logger = get_neat_logger(path=LOGS_PATH)

### Configuration

In [7]:
config_file = 'classification-miso'
config = create_configuration(filename=f'/{config_file}.json')

In [9]:
# TODO: better mechanism for override
config.pop_size = 150
# config.n_samples = 50
#
config.n_generations = 300
config.node_add_prob = 0.5
config.n_output = 2
config.n_processes = 10


ALGORITHM_VERSION = 'bayes-neat'
DATASET = config_file
# CORRELATION_ID = 'parameters_grid'
CORRELATION_ID = 'test'

In [10]:
report_repository = ReportRepository.create(project='neuro-evolution', logs_path=LOGS_PATH)
notifier = SlackNotifier.create(channel='batch-jobs')

In [13]:
evaluation_engine.n_processes

10

### Initialize

In [40]:
report = EvolutionReport(report_repository=report_repository,
                         algorithm_version=ALGORITHM_VERSION,
                         dataset=DATASET,
                         correlation_id=CORRELATION_ID)

In [41]:
population_engine = PopulationEngine(stagnation_engine=Stagnation())
speciation_engine = SpeciationEngine()
evaluation_engine = EvaluationStochasticEngine()

In [42]:
end_condition = 'normal'
# try:
# initialize population
population = population_engine.initialize_population()
speciation_engine.speciate(population, generation=0)

population = evaluation_engine.evaluate(population=population)

# report
report.report_new_generation(generation=0,
                                  population=population,
                                  species=speciation_engine.species)

2020-03-22 10:57:19,827 - INFO - Generation 0. Best fitness: -1.12. N-Parameters Best: 8
2020-03-22 10:57:19,830 - INFO -                          Mean fitness: -1.783. Mean N-Parameters: 8.0
2020-03-22 10:57:19,830 - INFO -     New best individual (77) found with fitness -1.12
Confusion Matrix:
[[129954 128546]
 [ 57939 183561]]
Accuracy: 62.702999999999996 %


In [43]:
def run_generation(generation, population):
    
    # create new generation's population
    population = population_engine.reproduce(species=speciation_engine.species,
                                             pop_size=population_engine.pop_size,
                                             generation=generation)
    # create new species based on new population
    speciation_engine.speciate(population, generation=generation)

    # evaluate
    population = evaluation_engine.evaluate(population=population)

    # generation report
    report.report_new_generation(generation=generation,
                                 population=population,
                                 species=speciation_engine.species)
    generation += 1
    return population

In [49]:
def show_species(species):
    species_keys = list(species.keys())
    representative_fitnesses = [round(specie.representative.fitness, 3) for specie in species.values()]
    best_fitnesses = [round(specie.fitness, 3) for specie in species.values()]

    n_genomes = [len(specie.members) for specie in species.values()]
    
    print(f'      keys: {species_keys}')
    print(f'      representative fitnesses: {representative_fitnesses}')
    print(f'      best fitness: {best_fitnesses}')
    print(f'      n_members: {n_genomes}')

In [45]:
generation = 0

In [46]:
for i in range(50):
    generation += 1
    population = run_generation(generation, population)
    show_species(species=speciation_engine.species)

2020-03-22 10:57:36,385 - INFO - Generation 1. Best fitness: -0.627. N-Parameters Best: 10
2020-03-22 10:57:36,387 - INFO -                          Mean fitness: -1.504. Mean N-Parameters: 10.493
2020-03-22 10:57:36,388 - INFO -     New best individual (161) found with fitness -0.627
Confusion Matrix:
[[187062  59438]
 [ 59545 193955]]
Accuracy: 76.2034 %
      keys: [1, 2]
      fitnesses: [-2.222, -1.957]
      n_members: [56, 94]
2020-03-22 10:57:50,615 - INFO - Generation 2. Best fitness: -0.518. N-Parameters Best: 10
2020-03-22 10:57:50,626 - INFO -                          Mean fitness: -1.243. Mean N-Parameters: 13.253
2020-03-22 10:57:50,627 - INFO -     New best individual (414) found with fitness -0.518
Confusion Matrix:
[[195666  50234]
 [ 48500 205600]]
Accuracy: 80.2532 %
      keys: [1, 2]
      fitnesses: [-1.126, -0.831]
      n_members: [46, 104]
2020-03-22 10:58:05,005 - INFO - Generation 3. Best fitness: -0.261. N-Parameters Best: 12
2020-03-22 10:58:05,010 - INFO -

In [50]:
show_species(species=speciation_engine.species)

      keys: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
      representative fitnesses: [-0.912, -0.346, -1.292, -2.045, -3.035, -1.019, -2.244, -2.931, -1.637, -1.265, -0.644, -1.762, -0.569, -0.897, -0.453, -1.503]
      best fitness: [-0.241, -0.472, -0.237, -0.804, -0.419, -0.237, -0.323, -0.732, -0.452, -0.31, -0.551, -0.372, -0.394, -0.298, -0.566, -0.305]
      n_members: [16, 2, 8, 8, 7, 24, 35, 10, 8, 6, 1, 12, 1, 2, 1, 6]


In [33]:
show_species(species=speciation_engine.species)

      keys: [1, 2]
      fitnesses: [-1.402, -0.747]
      n_members: [69, 81]


In [19]:
speciation_engine.species[1].representative.fitness

-1.640533208847046

In [20]:
speciation_engine.species[2].representative.fitness

-3.7541580200195312

### Check evaluation results

In [31]:
from neat.evaluation.evaluate_simple import evaluate_genome
from neat.evaluation.utils import get_dataset
from neat.loss.vi_loss import get_loss, get_beta

In [29]:
testing = False

In [33]:
dataset = get_dataset(config.dataset, testing=testing)
loss = get_loss(problem_type=config.problem_type)

GETTING DATASET
